In [7]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re

def setup_firefox_driver():
    """配置Firefox浏览器驱动"""
    # 设置Firefox选项
    firefox_options = Options()
    
    # 创建Firefox驱动实例
    driver = webdriver.Firefox(options=firefox_options)
    
    return driver

def scrape_fang_listings(driver, url, pages=2):
    """爬取房天下租房信息"""
    all_listings = []
    
    try:
        driver.get(url)
        print(f"正在访问: {url}")
        time.sleep(3)  # 等待页面加载
        
        for page in range(1, pages + 1):
            print(f"正在爬取第 {page} 页...")
            
            # 等待房源列表加载
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.houseList"))
            )
            
            # 获取所有房源元素 - 根据提供的HTML结构调整
            listings = driver.find_elements(By.CSS_SELECTOR, "dl.list")
            
            for listing in listings:
                try:
                    # 获取房源ID
                    dt_element = listing.find_element(By.CSS_SELECTOR, "dt")
                    house_id = dt_element.get_attribute("id").replace("rentid_", "").replace("_01", "")
                    
                    # 提取标题
                    title_element = listing.find_element(By.CSS_SELECTOR, "p.title a")
                    title = title_element.text.strip()
                    link = title_element.get_attribute("href")
                    
                    
                    # 提取font15中的详细信息（租赁方式、户型、面积、朝向）
                    try:
                        # 找到font15元素并获取完整文本
                        font15_element = listing.find_element(By.CSS_SELECTOR, "p.font15.mt12.bold")
                        font15_text = font15_element.text.strip()
                        
                        # 根据竖线分割信息
                        info_parts = font15_text.split('|')
                        
                        # 提取各个部分
                        rent_type = info_parts[0].strip() if len(info_parts) > 0 else "未知"
                        house_type = info_parts[1].strip() if len(info_parts) > 1 else "未知"
                        area = info_parts[2].strip() if len(info_parts) > 2 else "未知"
                        orientation = info_parts[3].strip() if len(info_parts) > 3 else "未知"
                        
                    except Exception as e:
                        rent_type = "未知"
                        house_type = "未知"
                        area = "未知"
                        orientation = "未知"
                        print(f"提取font15详细信息时出错: {e}")
                    
                    # 获取地址
                    try:
                        address_elements = listing.find_elements(By.CSS_SELECTOR, "p.gray6")
                        address = " ".join([elem.text.strip() for elem in address_elements if elem.text.strip()])
                    except:
                        address = "地址不可用"
                    
                    # 获取价格
                    try:
                        price_element = listing.find_element(By.CSS_SELECTOR, "div.moreInfo span.price")
                        price = price_element.text.strip()
                    except:
                        try:
                            # 尝试其他可能的价格选择器
                            price_elements = listing.find_elements(By.CSS_SELECTOR, "p.mt15")
                            price = " ".join([elem.text for elem in price_elements if elem.text.strip()])
                        except:
                            price = "价格不可用"
                    
                    # 打印调试信息
                    print(f"调试信息 - font15原始文本: {font15_text}")
                    print(f"调试信息 - 分割后: {info_parts}")
                    print(f"调试信息 - 提取结果: 租赁方式={rent_type}, 户型={house_type}, 面积={area}, 朝向={orientation}")
                    
                    # 整理数据
                    listing_data = {
                        "房源ID": house_id,
                        "标题": title,
                        "链接": link,
                        "租赁方式": rent_type,
                        "户型": house_type,
                        "面积": area,
                        "朝向": orientation,
                        "地址": address,
                        "价格": price,
                    }
                    
                    all_listings.append(listing_data)
                    print(f"已爬取: {title} - {rent_type} {house_type} {area} {orientation}")
                    
                except Exception as e:
                    print(f"提取房源信息时出错: {e}")
            
            # 检查是否有下一页并点击 (如果当前页小于总页数)
            if page < pages:
                try:
                    # 尝试定位并点击下一页按钮
                    next_page = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '下一页')]"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", next_page)
                    next_page.click()
                    time.sleep(3)  # 等待新页面加载
                except Exception as e:
                    print(f"点击下一页按钮时出错或已到最后一页: {e}")
                    break
    
    except Exception as e:
        print(f"爬虫运行时出错: {e}")
    
    return all_listings

def save_to_csv(data, filename="房天下租房数据.csv"):
    """将爬取的数据保存为CSV文件"""
    if not data:
        print("没有数据可保存")
        return
    
    keys = data[0].keys()
    
    with open(filename, 'w', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"数据已保存到 {filename}")

def main():
    url = "https://zu.fang.com/house-a010-b05048/"
    driver = setup_firefox_driver()
    
    try:
        listings = scrape_fang_listings(driver, url, pages=2)  # 爬取2页
        save_to_csv(listings)
    finally:
        driver.quit()
        print("浏览器已关闭")

if __name__ == "__main__":
    main()

正在访问: https://zu.fang.com/house-a010-b05048/
正在爬取第 1 页...
调试信息 - font15原始文本: 整租|1室1厅|41㎡|朝东
调试信息 - 分割后: ['整租', '1室1厅', '41㎡', '朝东']
调试信息 - 提取结果: 租赁方式=整租, 户型=1室1厅, 面积=41㎡, 朝向=朝东
已爬取: 实图实价 公寓前台 不存在租后找不到人 非全新无甲醛 - 整租 1室1厅 41㎡ 朝东
调试信息 - font15原始文本: 合租主卧|2户合租|15㎡|朝南
调试信息 - 分割后: ['合租主卧', '2户合租', '15㎡', '朝南']
调试信息 - 提取结果: 租赁方式=合租主卧, 户型=2户合租, 面积=15㎡, 朝向=朝南
已爬取: 无服务费 合生世界村H区 上面次卧带阳台 电梯房 3 - 合租主卧 2户合租 15㎡ 朝南
调试信息 - font15原始文本: 合租主卧|2户合租|25㎡|朝南
调试信息 - 分割后: ['合租主卧', '2户合租', '25㎡', '朝南']
调试信息 - 提取结果: 租赁方式=合租主卧, 户型=2户合租, 面积=25㎡, 朝向=朝南
已爬取: 无服务费 合生I区 超大卧室 南向的 齐全 随时 - 合租主卧 2户合租 25㎡ 朝南
调试信息 - font15原始文本: 合租主卧|2户合租|20㎡|朝南
调试信息 - 分割后: ['合租主卧', '2户合租', '20㎡', '朝南']
调试信息 - 提取结果: 租赁方式=合租主卧, 户型=2户合租, 面积=20㎡, 朝向=朝南
已爬取: 无服务费 晶彩亦庄 超大卧室 电梯房 可以提前看房预 - 合租主卧 2户合租 20㎡ 朝南
调试信息 - font15原始文本: 合租主卧|2户合租|20㎡|朝南
调试信息 - 分割后: ['合租主卧', '2户合租', '20㎡', '朝南']
调试信息 - 提取结果: 租赁方式=合租主卧, 户型=2户合租, 面积=20㎡, 朝向=朝南
已爬取: 无服务费 橡树湾北区 三居室 主卧出租 南向4月初可 - 合租主卧 2户合租 20㎡ 朝南
调试信息 - font15原始文本: 整租|2室1厅|85㎡|朝南
调试信息 - 分割后: ['整租', '2室1厅', '85㎡', '朝南'